In [22]:
# Import packages -----------------------------------
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import calendar
# from dateutil import parser
import gspread
import urllib.request, json 

import plotly.express as px
import plotly.graph_objects as go
# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, State, callback
import dash_bootstrap_components as dbc
# from dash_extensions import Lottie
from dash import dash_table

import sys

sys.path.append(".")
# from assets.googleService import getDataframe_City
# from assets.FB_audienceMetrics import audienceTimeOfDayDetail

# Import Dataset --------------------------------------------------
sa = gspread.service_account(filename="Assets\zyp-art-gallery-62e00b2be4ff.json");

def getDataframe_City(sheet, worksheet):
    global sa
    sh = sa.open(sheet);
    wks = sh.worksheet(worksheet);
    # dataframe = pd.DataFrame(wks.get_all_records());
    listOfLists = wks.get_all_values();
    return listOfLists;

sheet = "ZypFacebook_Audience-CanadianCity1";
worksheet = "ZypFacebook_Audience-CanadianCity1";
listOfLists = getDataframe_City(sheet, worksheet);

# Prepare Data --------------------------------------------
df1 = pd.DataFrame(listOfLists[1:], columns=listOfLists[0])
df1["end_time"] = pd.to_datetime(df1["end_time"]);

In [23]:
# Province Reference: https://geogratis.gc.ca/services/geoname/en/codes/province
province = [
    {
        "code":10,
        "term":"NL",
        "definition":"Newfoundland and Labrador"
    },
    {
        "code":11,
        "term":"PE",
        "definition":"Prince Edward Island"
    },
    {
        "code":12,
        "term":"NS",
        "definition":"Nova Scotia"
    },
    {
        "code":13,
        "term":"NB",
        "definition":"New Brunswick"
    },
    {
        "code":24,
        "term":"QC",
        "definition":"Quebec"
    },
    {
        "code":35,
        "term":"ON",
        "definition":"Ontario"
    },
    {
        "code":46,
        "term":"MB",
        "definition":"Manitoba"
    },
    {
        "code":47,
        "term":"SK",
        "definition":"Saskatchewan"
    },
    {
        "code":48,
        "term":"AB",
        "definition":"Alberta"
    },
    {
        "code":59,
        "term":"BC",
        "definition":"British Columbia"
    },
    {
        "code":60,
        "term":"YU",
        "definition":"Yukon"
    },
    {
        "code":61,
        "term":"NT",
        "definition":"Northwest Territories"
    },
    {
        "code":62,
        "term":"NU",
        "definition":"Nunavut"
    },
];

In [24]:
# Concise Reference: https://geogratis.gc.ca/services/geoname/en/codes/concise
# concise = ["CITY","TOWN","VLG","HAM","UTM","LTM","STM","MUN1","MUN2","UNP","IR","GEOG","PARK","MIL","ROAD"]
concise = [
    {
        "code":"CITY",
        "term":"City",
        "definition":"Populated place with legally defined boundaries, usually incorporated under a provincial or territorial Municipal Act and being the highest level of municipal incorporation (city, cité, ville)"
    },
    {
        "code":"TOWN",
        "term":"Town",
        "definition":"Populated place with legally-defined boundaries, usually incorporated under a provincial Municipal Act, of lesser status than a city (town, borough, separated town, municipality, municipalité de village)"
    }
];

In [25]:
selected = df1.loc[10, "end_time"];
mask = df1["end_time"] == selected;

In [26]:
df1[mask]

,end_time,"Calgary, AB, Canada","Patterson Place, AB, Canada","Eastview, AB, Canada","Hounsfield Heights, AB, Canada","Country Club Estates, Grande Prairie, Alberta, AB, Canada","Goodfish Lake, AB, Canada","Blackburne, Edmonton, AB, Canada","Ellscott, AB, Canada","Finnegan, AB, Canada",...,"Aishihik, YT, Canada","Moosehide, YT, Canada","Braeburn, YT, Canada","M'Clintock, YT, Canada","Canyon, YT, Canada","Champagne, YT, Canada","Black Hills, YT, Canada","Minto, YT, Canada","Livingstone Creek, YT, Canada","Lower La Barge, YT, Canada"
10,2022-05-21,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df = df1[mask];
index = df[mask].index.values.astype(int)[0];

cities = df.columns.tolist()[1:];

data = [];
for i in range(len(cities)):
    row = [];
    count = int(df[mask].transpose()[1:][index][i]);
    if count > 0:
        row.append(cities[i]);
        row.append(count);
        data.append(row);
dfTemp = pd.DataFrame(data, columns=["City","Count"]);
# dfTemp = dfTemp.sort_values(["Count","City"], ascending=False);
dfTemp

C:\Users\kunal\AppData\Local\Temp/ipykernel_15956/2145450089.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = df[mask].index.values.astype(int)[0];
C:\Users\kunal\AppData\Local\Temp/ipykernel_15956/2145450089.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  count = int(df[mask].transpose()[1:][index][i]);


,City,Count
0,"Calgary, AB, Canada",12
1,"Edmonton, AB, Canada",80
2,"Bon Accord, AB, Canada",1
3,"Rocky Mountain House, AB, Canada",1
4,"Nisku, AB, Canada",2
5,"Onoway, AB, Canada",2
6,"Whitecourt, AB, Canada",1
7,"Saint Albert, AB, Canada",2
8,"Grande Prairie, AB, Canada",2
9,"Calmar, AB, Canada",40


In [28]:
provinceAvailable = [];
for i in dfTemp.index:
    cityValue = dfTemp.loc[i, "City"];
    provinceAbberviation = cityValue.split(", ")[1];
    provinceAvailable.append(provinceAbberviation);
provinceAvailable = list(dict.fromkeys(provinceAvailable));
provinceAvailable

['AB', 'BC', 'MB', 'NL', 'ON']

In [40]:
# labels = ["City","Count","Latitude","Longitude"]
data = [];
for i in range(len(province)):
    for j in range(len(concise)):
        for x in dfTemp.index:
            if province[i].get("term") == dfTemp.loc[x, "City"].split(", ")[1]:
                provinceCode = str(province[i].get("code"));
                # print(provinceCode);
                conciseCode = concise[j].get("code");
                # print(conciseCode);
                with urllib.request.urlopen(f"https://geogratis.gc.ca/services/geoname/en/geonames.json?&province={provinceCode}&concise={conciseCode}") as url:
                    cityData = json.loads(url.read().decode());
                # row = [];
                for y in range(len(cityData["items"])):
                    # row = [];
                    if dfTemp.loc[x, "City"].split(", ")[0] == cityData["items"][y].get("name"):
                        row = [];
                        # print(dfTemp.loc[x, "City"].split(", ")[0]);
                        # print(cityData["items"][y].get("name"));
                        row.append(dfTemp.loc[x, "City"]);
                        row.append(dfTemp.loc[x, "Count"]);
                        row.append(cityData["items"][y].get("latitude"));
                        row.append(cityData["items"][y].get("longitude"));
                        row.append(province[i].get("definition"));
                        # print(row);
                        data.append(row);
                    
dfFinal = pd.DataFrame(data, columns=["Location","Count","Latitude","Longitude","Province"]);
dfFinal = dfFinal.sort_values(["Count","Location"], ascending=False);
dfFinal

,Location,Count,Latitude,Longitude,Province
4,"Edmonton, AB, Canada",80,53.534444,-113.490278,Alberta
16,"Calmar, AB, Canada",40,53.257778,-113.810833,Alberta
7,"Leduc, AB, Canada",27,53.266667,-113.545833,Alberta
17,"Devon, AB, Canada",24,53.362222,-113.722500,Alberta
3,"Calgary, AB, Canada",12,51.045833,-114.057500,Alberta
6,"Wetaskiwin, AB, Canada",4,52.969167,-113.376944,Alberta
11,"Spruce Grove, AB, Canada",4,53.553889,-113.902222,Alberta
24,"Burnaby, BC, Canada",3,49.242778,-122.972778,British Columbia
20,"Vancouver, BC, Canada",2,49.261111,-123.113889,British Columbia
5,"Grande Prairie, AB, Canada",2,55.170833,-118.796111,Alberta


In [41]:
dfFinal.to_csv("TestCity.csv", index=False)